In [ ]:
from pathlib import Path
import tomllib
import numpy as np
from scipy import constants as C
from mmwave.rma import rma, echo_plot
from mmwave import schemas

In [ ]:
output_dir = Path("/home/tomin/python/mmwave_postproc/outdoor_20241121_152633")
output_dir = Path("/home/tomin/python/mmwave_postproc/outdoor_20241121_143316")

frame_file_path = output_dir / "all_mmw_array.npy"
frame_file: np.ndarray = np.load(frame_file_path, mmap_mode="r")
print(frame_file.shape)


with (output_dir / "config.toml").open("rb") as f:
    cfg = tomllib.load(f)
    cfg = schemas.MMWConfig.model_validate(cfg)

num_sample = cfg.mimo.profile.numAdcSamples
nFFTtime = num_sample  # Number of FFT points for Spatial-FFT
adcStartTime = cfg.mimo.profile.adcStartTime  # us
startFrequency = cfg.mimo.profile.startFrequency * 1e9
K = cfg.mimo.profile.frequencySlope * 1e12  # Slope const (hz/s)
Fs = cfg.mimo.profile.adcSamplingFrequency * 1e3  # Sampling rate (sps)

F0 = startFrequency + adcStartTime * K * 1e-6 + num_sample // 2 / Fs * K  # Center frequency
F0_min = startFrequency + adcStartTime * K * 1e-6
F0_max = startFrequency + adcStartTime * K * 1e-6 + num_sample / Fs * K
print(F0_min * 1e-9, F0 * 1e-9, F0_max * 1e-9, num_sample / Fs * K * 1e-9)

dx = cfg.bracket.profile.dx
dy = cfg.bracket.profile.dy  # Sampling distance at x (horizontal) y (vertical) axis in mm
row = cfg.bracket.profile.row
col = cfg.bracket.profile.col
print(dx, dy, row, col)
rw = dx * col / 1000
rh = dy * row / 1000

pic_positions = np.mgrid[0 : rh : 1j * 1001, 0 : rw : 1j * 501].transpose(1, 2, 0)

h, w = pic_positions.shape[:2]
print(rh, rw, h, w)

c = C.c
Ts = 1 / Fs  # Sampling period
k = 2 * np.pi * F0 / c  # Wave number
print(f"k:{k}")

R_step = c / 2 / (K * Ts * num_sample)
print(R_step * 1000)  # mm

In [ ]:
tx_idx = 1
rx_idx = 1
ID_select = 21
tI = 183  # mm
R = c / 2 * (ID_select / (K * Ts * nFFTtime)) - tI / 1000

Echo = frame_file[:, :, tx_idx, rx_idx, :, 0] + 1j * frame_file[:, :, tx_idx, rx_idx, :, 1]
Sr: np.ndarray = np.fft.fft(Echo)
Sr = Sr[:, :, ID_select - 1]
Sr[Sr == 0] = 1e-10
echo_plot(Sr, "source", dx, dy)

In [ ]:
reconstructed_image = rma(Sr, dx, dy, R, k)
echo_plot(reconstructed_image, "reconstructed_image", dx, dy, rma=True)